In this notebook, we are trying to reproduce [the paper](https://pdfs.semanticscholar.org/edff/b62b32ffcc2b5cc846e26375cb300fac9ecc.pdf) for speaker change detection

## Review

**Sequence Labelling** 

They think this task as a binary classification. Thus, they label changing frame as a **1** and non-changing frame as a **0**. So that, they use the _binary cross-entropy loss function_.

**Network Architecture**
- 2 Bi-LSTM
    - 64 and 32 outputs respectively.
- Multi Layer Perceptron
    - 3 Fully Connected Feedforward Layers
        - 40, 20, 1 dimensional respectively.
    - Tanh activation for first 2 layer
    - Sigmoid activation for last layer
    
**Feature Extraction**
- "35-dimensional acoustic features are extracted every 16ms on a 32ms window using [Yaafe toolkit](http://yaafe.sourceforge.net)."
    - 11 Mel-Frequency Cepstral Coefficients (MFCC), 
    - Their first and second derivatives,
    - First and second derivatives of the energy.

**Class Imbalance**

- _"The number of positive labels isincreased artificially by labeling as positive every frame in the direct neighborhood of the manually annotated change point."_
- A positive neighborhood of 100ms (50ms on both sides) is used around each change point, to partially solve the class imbalance problem.

**Subsequences**

- _"The long audio sequences are split into short fixed-length overlapping sequences."_

**Prediction**

- _"Finally, local score maxima exceeding a pre-determined threshold θ are marked as speaker change points."_

**Training**

- Subsequences for training are 3.2s long with a step of 800ms (i.e. two adjacent sequences overlap by 75%).

## Code

### Feature Extraction

We will use Yaafe Toolkit. (To see all available features, you can use _!yaafe -l_) To learn how we can do that, start with http://yaafe.github.io/Yaafe/manual/quickstart.html#quick-start-using-yaafe


In [ ]:
# You can view a description of each feature (or output format) with the -d option:

!yaafe -d MFCC

In [ ]:
!yaafe -d Energy

Let's determine blockSize and stepSize. 

If we have 16kHz audio signal(in AMI, we have 16kHz), for 32 ms block, we need 16x32, For the stepsize as 16 ms, we need 16x16 size.

We need these features:

- mfcc: MFCC blockSize=512 stepSize=256 CepsNbCoeffs=11
- mfcc_d1: MFCC blockSize=512 stepSize=256 CepsNbCoeffs=11 > Derivate DOrder=1
- mfcc_d2: MFCC blockSize=512 stepSize=256 CepsNbCoeffs=11 > Derivate DOrder=2
- energy_d1: Energy blockSize=512 stepSize=256  > Derivate DOrder=1
- energy_d2: Energy blockSize=512 stepSize=256  > Derivate DOrder=2

To extract all of these, we will use [this technique](http://yaafe.github.io/Yaafe/manual/quickstart.html#extract-several-features). Shortly, we will write all these features into single text file.

In [ ]:
f = open("featureplan.txt", "w")
f.write("mfcc: MFCC blockSize=512 stepSize=256 CepsNbCoeffs=11 \n"
        "mfcc_d1: MFCC blockSize=512 stepSize=256 CepsNbCoeffs=11 > Derivate DOrder=1 \n"
        "mfcc_d2: MFCC blockSize=512 stepSize=256 CepsNbCoeffs=11 > Derivate DOrder=2 \n"
        "energy_d1: Energy blockSize=512 stepSize=256  > Derivate DOrder=1 \n"
        "energy_d2: Energy blockSize=512 stepSize=256  > Derivate DOrder=2")
f.close() 

In [ ]:
cat featureplan.txt

In [ ]:
#!ls

In [ ]:
!yaafe -c featureplan.txt -r 16000 a2002011001-e02-16kHz.wav -p Precision=8 -p Metadata=False

**With Librosa**

In [1]:
import librosa
import os
import glob
import numpy as np
import sys

def wav_to_matrix(filename, hop, win_len): # hop and win_len in milisecond 
    audio, sr = librosa.load(filename)
    # https://github.com/librosa/librosa/issues/584
    mfccs = librosa.feature.mfcc(audio, sr, n_mfcc=11, hop_length=int(float(hop/1000)*sr), n_fft=int(float(win_len/1000)*sr))
    mfccs_d1 = librosa.feature.delta(mfccs)
    mfccs_d2 = librosa.feature.delta(mfccs, order=2)
    energy = librosa.feature.rmse(y=audio, hop_length=int(float(hop/1000)*sr), frame_length=int(float(win_len/1000)*sr))
    energy_d1 = librosa.feature.delta(energy)
    energy_d2 = librosa.feature.delta(energy, order=2)
    print (mfccs.shape)
    print (mfccs_d1.shape)
    print (mfccs_d2.shape)
    print (energy_d1.shape)
    print (energy_d2.shape)

    a = np.vstack((mfccs, mfccs_d1, mfccs_d2, energy_d1, energy_d2))
    # line_mfccs = np.ravel(mfccs, order='F')
    return a

In [34]:
import math
import matplotlib.pyplot as pp

%matplotlib inline


def create_data_for_supervised(root_dir, hop, win_len, from_ep = 0, to_ep=0, boost_for_imbalance=False):
    all_audio_paths = glob.glob(os.path.join(root_dir, '*wav'))
    matrix_of_all_audio = []
    
    output_array = []
    
    num = 0
    
    for single_audio_path in all_audio_paths:
        num += 1
        
        if ((num >= from_ep) and (num < to_ep)):
            
            end_time_array_second = []

            filename = (single_audio_path.split("/")[-1]).split(".")[0]
            matrix_of_single_audio = wav_to_matrix(single_audio_path, hop, win_len)
            array_of_single_audio = np.ravel(matrix_of_single_audio)

            if (matrix_of_single_audio is not None):

                print (matrix_of_single_audio.shape)
                matrix_of_all_audio.extend(array_of_single_audio)
                print (single_audio_path + " is done.")

                main_set = "./txt_files/" + filename + "_end_time.txt"# FILENAME PATH for TXT

                with open(main_set) as f:
                    content = f.readlines()

                # you may also want to remove whitespace characters like `\n` at the end of each line


                # need to open text file
                # after that, point the end point of speaker
                # add 1 to point of speaker, add 0 otherwise
                # time is in second format at the txt file
                content = [x.strip() for x in content] 

                for single_line in content:

                    end_time_array_second.append(single_line)

                    # we use following method to get milisecond version
                    # float(win_len + ((offset+100) * hop)) 
                    # we need to inversion of that

                output_array = np.zeros(matrix_of_single_audio.shape[1])

                for end_time in end_time_array_second:
                    end_time_ms = float(end_time)*1000
                    which_start_hop = (end_time_ms-win_len)/hop # now we know, milisecond version of change
                                                # which is located after which_hop paramater
                                                # add 2 and round to up
                    which_end_hop = end_time_ms/hop # round to up
                    
                    start_location = math.ceil(which_start_hop + 1)
                    end_location = math.ceil(which_end_hop)

                    # print ("s:", start_location)
                    # print ("e:", end_location)
                    if (boost_for_imbalance==False):
                        output_array[start_location:end_location+1] = 1.0
                        
                    else:
                        output_array[start_location-2:end_location+3] = 1.0


            # print (output_array)
            print (output_array.mean())
            # ar = np.arange(matrix_of_single_audio.shape[1]) # just as an example array
            # pp.plot(ar, output_array, 'x')
            # pp.show()
                
            
    audio_array = np.asarray(matrix_of_all_audio)
    audio_array = np.reshape(matrix_of_all_audio, (35,-1))
   
        
    input_array = np.asarray(audio_array)
    input_array = input_array.reshape((len(input_array), np.prod(input_array.shape[1:])))  
    print(input_array.shape)
    
    output_array = np.asarray(output_array)
    output_array = np.expand_dims(output_array, axis=0)
    print(output_array.shape)

    return (input_array, output_array)

In [35]:
inn, out = create_data_for_supervised ("./amicorpus/*/audio/", 16, 32, 0, 2, True)

(11, 154884)
(11, 154884)
(11, 154884)
(1, 154884)
(1, 154884)
(35, 154884)
./amicorpus/IN1002/audio/IN1002.Mix-Headset.wav is done.
0.025238242814
(35, 154884)
(1, 154884)


In [ ]:
!pwd

In [ ]:
k = wav_to_matrix("How to Read a Research Paper.mp3", 16, 32)

In [ ]:
k.shape

In [ ]:
k = wav_to_matrix("How to Read a Research Paper.mp3", 32, 32)

In [ ]:
import matplotlib.pyplot as pp
%matplotlib inline

pp.plot(np.swapaxes(k, 0, 1))
pp.axhline(y=0.5, color='r', linestyle='-')
pp.show()

### Create Subsequences with Label

At that point, we should create training and test data with their label. Also, we can use directly [pyannote.metrics](https://github.com/pyannote/pyannote-metrics)

### Deep Learning Architecture

We can directly upload the model's architecture from the .yml file which is provided by writer.

However, I want to directly write all steps.

In [ ]:
# Author's .yml files

!wget https://raw.githubusercontent.com/yinruiqing/change_detection/master/model/architecture.yml

In [16]:
# Load to model

from keras.models import model_from_yaml
yaml_file = open('architecture.yml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
model = model_from_yaml(loaded_model_yaml)

/home/herdogan/anaconda3/envs/pyannote/lib/python3.6/site-packages/keras/engine/topology.py:1271: UserWarning: Update your `InputLayer` call to the Keras 2 API: `InputLayer(batch_input_shape=(None, 320..., name="labeling_input", sparse=False, dtype="float32")`
  return cls(**config)
/home/herdogan/anaconda3/envs/pyannote/lib/python3.6/site-packages/keras/layers/recurrent.py:2209: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  return cls(**config)
/home/herdogan/anaconda3/envs/pyannote/lib/python3.6/site-packages/keras/layers/recurrent.py:2209: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(activation="tanh", batch_input_shape=(None, 320..., go_backwards=False, input_dtype="float32", name="forward_lstm_0", return_sequences=True, stateful=False, trainable=True, unroll=False, unit_forget_bias=True, input_shape=(None, 35), units=32, kernel_initializer="glorot_uniform", recurrent_initializer="orthogo

In [ ]:
import keras
print (keras.__version__)

In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
labeling_input (InputLayer)  (None, 320, 35)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 320, 64)           17408     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 320, 40)           13600     
_________________________________________________________________
timedistributed_1 (TimeDistr (None, 320, 40)           1640      
_________________________________________________________________
timedistributed_2 (TimeDistr (None, 320, 10)           410       
_________________________________________________________________
timedistributed_3 (TimeDistr (None, 320, 1)            11        
Total params: 33,069
Trainable params: 33,069
Non-trainable params: 0
_________________________________________________________________


In [26]:
from keras import layers
from keras import models
from keras import optimizers
import keras
from keras.models import Model
import tensorflow as tf
from keras.layers.advanced_activations import *
from keras.utils.generic_utils import get_custom_objects


frame_shape = (320, 35)

## Network Architecture

input_frame = keras.Input(frame_shape, name='main_input')

bidirectional_1 = layers.Bidirectional(layers.LSTM(32, return_sequences=True))(input_frame)
bidirectional_2 = layers.Bidirectional(layers.LSTM(20, activation='tanh', return_sequences=True))(bidirectional_1)

tdistributed_1 = layers.TimeDistributed(layers.Dense(40, activation='tanh'))(bidirectional_2)
tdistributed_2 = layers.TimeDistributed(layers.Dense(10, activation='tanh'))(tdistributed_1)
tdistributed_3 = layers.TimeDistributed(layers.Dense(1, activation='sigmoid'))(tdistributed_2)

## Source: https://stackoverflow.com/questions/37743574/hard-limiting-threshold-activation-function-in-tensorflow
def step_activation(x):
    threshold = 0.4
    cond = tf.less(x, tf.fill(value=threshold, dims=tf.shape(x)))
    out = tf.where(cond, tf.zeros(tf.shape(x)), tf.ones(tf.shape(x)))

    return out

step_activation = layers.Dense(1, activation=step_activation, name='threshold_activation')(tdistributed_3)



model = Model(input_frame, step_activation)

rmsprop = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)

model.compile(loss='binary_crossentropy', optimizer="rmsprop")

In [27]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (None, 320, 35)           0         
_________________________________________________________________
bidirectional_17 (Bidirectio (None, 320, 64)           17408     
_________________________________________________________________
bidirectional_18 (Bidirectio (None, 320, 40)           13600     
_________________________________________________________________
time_distributed_25 (TimeDis (None, 320, 40)           1640      
_________________________________________________________________
time_distributed_26 (TimeDis (None, 320, 10)           410       
_________________________________________________________________
time_distributed_27 (TimeDis (None, 320, 1)            11        
_________________________________________________________________
threshold_activation (Dense) (None, 320, 1)            2         
Total para

In [13]:
# To save our model

model_yaml = model.to_yaml()
with open("model.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)

In [14]:
# To look our model

!cat model.yaml

backend: tensorflow
class_name: Model
config:
  input_layers:
  - [main_input, 0, 0]
  layers:
  - class_name: InputLayer
    config:
      batch_input_shape: !!python/tuple [null, 320, 35]
      dtype: float32
      name: main_input
      sparse: false
    inbound_nodes: []
    name: main_input
  - class_name: Bidirectional
    config:
      layer:
        class_name: LSTM
        config:
          activation: tanh
          activity_regularizer: null
          bias_constraint: null
          bias_initializer:
            class_name: Zeros
            config: {}
          bias_regularizer: null
          dropout: 0.0
          go_backwards: false
          implementation: 1
          kernel_constraint: null
          kernel_initializer:
            class_name: VarianceScaling
            config: {distribution: uniform, mode: fan_avg, scale: 1.0, seed: null}
          kernel_regularizer: null
          name: lstm_7
          recurrent_activation: hard_sigmoid
          recurrent_constr

In [37]:
from keras.models import load_model
model.compile

how_many_step = 2
ix_step = 0
from_epi = 0

while (ix_step < how_many_step):
    ix_step += 1
    
    input_array, output_array = create_data_for_supervised ("./amicorpus/*/audio/", 16, 32, 0, 2, True)
    
    max_len = 320 # how many frame will be taken
    step = 240 # step size.

    input_array_specified = []
    output_array_specified = []

    for i in range (0, input_array.shape[1]-max_len, step):
        single_input_specified = np.transpose(input_array[:,i:i+max_len])
        single_output_specified = np.transpose(output_array[:,i+max_len])
        
        input_array_specified.append(single_input_specified)
        output_array_specified.append(single_output_specified)

    output_array_specified = np.asarray(output_array_specified)
    input_array_specified = np.asarray(input_array_specified)
    
    model.fit(input_array_specified, output_array_specified,
               epochs=2,
               batch_size=16,
               shuffle=True)
    
    # https://keras.io/getting-started/faq/#how-can-i-save-a-keras-model

    model.save_weights('bilstm_weights.h5')    
    
    input_array = []
    output_array = []
    
    from_epi += 4

(11, 154884)
(11, 154884)
(11, 154884)
(1, 154884)
(1, 154884)
(35, 154884)
./amicorpus/IN1002/audio/IN1002.Mix-Headset.wav is done.
0.025238242814
(35, 154884)
(1, 154884)


ValueError: Error when checking target: expected threshold_activation to have 3 dimensions, but got array with shape (645, 1)